# Reading SEM images in tiff format and create a Nexus file

In this notebook we define standard function to read tiff images and we try to build a 'Nexus' file.

In [68]:
!pip install tifffile
!pip install ncempy
!pip install nexusformat


In [ ]:
import numpy as np
import h5py
from nexusformat.nexus import *
from ncempy.io import dm
import datetime
from datetime import datetime
path_file='' # insert the path of this notebook

name_txt='600K-FFT.dm3'

In [57]:
raw_data = dm.dmReader(name_txt)
for k in raw_data.keys():
    print(k, ":", raw_data[k])

filename : 10K.dm3
data : [[1165 1142 1088 ...  977  970 1020]
 [1162 1125 1067 ...  993  970 1005]
 [1140 1198 1147 ...  948  959 1004]
 ...
 [ 357  363  347 ...  969  974  975]
 [ 356  358  353 ...  966  934  951]
 [ 358  355  357 ...  980  978  950]]
pixelUnit : ['µm', 'µm']
pixelSize : [0.0016042781, 0.0016042781]
coords : [array([0.0000e+00, 1.6000e-03, 3.2000e-03, ..., 1.6380e+00, 1.6396e+00,
       1.6412e+00]), array([0.0000e+00, 1.6000e-03, 3.2000e-03, ..., 1.6380e+00, 1.6396e+00,
       1.6412e+00])]


In [ ]:

img = raw_data['data']





dict_keys(['filename', 'data', 'pixelUnit', 'pixelSize', 'coords'])
Dimensioni immagine: (1024, 1024)

--- Metadata principali ---
filename : 10K.dm3
data : [[1165 1142 1088 ...  977  970 1020]
 [1162 1125 1067 ...  993  970 1005]
 [1140 1198 1147 ...  948  959 1004]
 ...
 [ 357  363  347 ...  969  974  975]
 [ 356  358  353 ...  966  934  951]
 [ 358  355  357 ...  980  978  950]]
pixelUnit : ['µm', 'µm']
pixelSize : [0.0016042781, 0.0016042781]
coords : [array([0.0000e+00, 1.6000e-03, 3.2000e-03, ..., 1.6380e+00, 1.6396e+00,
       1.6412e+00]), array([0.0000e+00, 1.6000e-03, 3.2000e-03, ..., 1.6380e+00, 1.6396e+00,
       1.6412e+00])]
coordinate0
1


[array([0.0000e+00, 1.6000e-03, 3.2000e-03, ..., 1.6380e+00, 1.6396e+00,
        1.6412e+00]),
 array([0.0000e+00, 1.6000e-03, 3.2000e-03, ..., 1.6380e+00, 1.6396e+00,
        1.6412e+00])]

You may extract the data as an array with

In [27]:
image_array = tifffile.imread(path_tiff+name_img)
image_array[0,0:30]

array([197, 192, 180, 191, 187, 196, 205, 188, 187, 185, 176, 181, 184,
       188, 195, 202, 209, 205, 199, 197, 198, 204, 199, 200, 197, 194,
       183, 187, 192, 186], dtype=uint8)

### Exercise 

Consider the NXem application definition https://fairmat-nfdi.github.io/nexus_definitions/classes/contributed_definitions/em-structure.html#em-structure. Choose some of the base classes that compone it and try to create a 'pseudo_nexus_em' starting from the SEM images at disposal.  

*Suggestion : try to map what you know about  a SEM  measurement with the base classes you have chosen.* 

In [ ]:
project='test'
buck_num='00'

filename = f"{project}-{buck_num}.nxs"
f = h5py.File(filename, "w")
f.attrs['default'] = 'entry'
f.close

<bound method File.close of <HDF5 file "test-00-20251120_094201.nxs" (mode r+)>>

In [72]:

test=nxload(filename)
print(test.tree)

root:NXroot
  @default = 'entry'


In [73]:
# /entry
f = h5py.File(filename, "r+")
f.create_group("entry")
f['/entry'].attrs["NX_class"] = "NXentry"
f['/entry'].attrs["default"] = "data"

In [ ]:
# /entry/definition
f['/entry'].create_dataset('definition',data='NXem')

# /entry/duration
f['/entry'].create_dataset('experiment_description',data='da definire')
f['/entry/experiment_description'].attrs["description"] = 'SEM'

<HDF5 dataset "title": shape (), type "|O">

In [75]:
f['/entry'].create_group("sample")
f['/entry/sample'].attrs["NX_class"] = "NXsample"

# /entry/sample/name
f['/entry/sample'].create_dataset('name',data='sample_name')

<HDF5 dataset "name": shape (), type "|O">

In [76]:
# /entry/user
f['/entry'].create_group("user")
f['/entry/user'].attrs["NX_class"] = "NXuser"

# \entry\user\name
f['/entry/user'].create_dataset('name',data='user_name')

<HDF5 dataset "name": shape (), type "|O">

In [77]:
# \entry\data --> group of the default plot
f['/entry'].create_group("data")
f['/entry/data'].attrs["NX_class"] = "NXdata"
f['/entry/data'].attrs["signal"] = "img"
#f['/entry/data'].attrs["axes"] = "energy"
f['/entry/data'].create_dataset('img',data=img)

<HDF5 dataset "img": shape (1024, 1024), type "<i2">

In [78]:
f.close()

In [79]:

test=nxload(filename)
print(test.tree)

root:NXroot
  @default = 'entry'
  entry:NXentry
    @default = 'data'
    data:NXdata
      @signal = 'img'
      img = int16(1024x1024)
    definition = 'NXem'
    experiment_description = 'da definire'
      @description = 'SEM'
    sample:NXsample
      name = 'sample_name'
    title = '2024_03_06_0005'
    user:NXuser
      name = 'user_name'
